In [4]:
import altair as alt
import pandas as pd
pd.set_option('display.max_columns', None)

In [8]:
migration_df = pd.read_csv('migration_flow.csv')
migration_df.sample(5)

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,CITIZENSHIP,Citizenship,FREQ,Frequency of observation,MEASURE,Measure,SEX,Sex,BIRTH_PLACE,Place of birth,EDUCATION_LEV,Education level,UNIT_MEASURE,Unit of measure,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals
176012,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,CZE,Czechia,MKD,North Macedonia,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2019.0,NaN,142.0,NaN,A,Normal value,0.0,Units,0,Zero
77797,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,SWE,Sweden,STP,Sao Tome and Principe,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2007.0,NaN,0.0,NaN,A,Normal value,0.0,Units,0,Zero
23025,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,NZL,New Zealand,EST,Estonia,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2004.0,NaN,2.0,NaN,A,Normal value,0.0,Units,0,Zero
135586,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,SVK,Slovak Republic,MAR,Morocco,A,Annual,B11,Inflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2005.0,NaN,3.0,NaN,A,Normal value,0.0,Units,0,Zero
209862,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,FIN,Finland,BLZ,Belize,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2020.0,NaN,0.0,NaN,A,Normal value,0.0,Units,0,Zero


In [ ]:
#cleanup:

columns_to_drop = [
    "STRUCTURE", "STRUCTURE_ID", "STRUCTURE_NAME", "ACTION",
    "FREQ", "Frequency of observation", "SEX", "Sex",
    "BIRTH_PLACE", "Place of birth", "EDUCATION_LEV", "Education level",
    "OBS_STATUS", "Observation status", "UNIT_MULT", "Unit multiplier",
    "DECIMALS", "Decimals", "UNIT_MEASURE", "Unit of measure", "Time period", "Observation value"
]

df_cleaned = migration_df.drop(columns=columns_to_drop)


In [13]:
df_cleaned.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
38010,HUN,Hungary,KAZ,Kazakhstan,B11,Inflows of foreign population,2022.0,729.0
201044,LUX,Luxembourg,BEL,Belgium,B12,Outflows of foreign population,1999.0,730.0
11061,FRA,France,AGO,Angola,B11,Inflows of foreign population,2008.0,819.0
212199,ISR,Israel,SUN_F,Former USSR,B11,Inflows of foreign population,2010.0,6999.0
104735,FRA,France,DOM,Dominican Republic,B11,Inflows of foreign population,2022.0,452.0


In [14]:

inflow_df = df_cleaned[df_cleaned['Measure'] == 'Inflows of foreign population']
inflow_df.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
99769,LUX,Luxembourg,CRI,Costa Rica,B11,Inflows of foreign population,2007.0,0.0
133775,FIN,Finland,TUN,Tunisia,B11,Inflows of foreign population,1998.0,19.0
133241,SWE,Sweden,LAO,Lao People’s Democratic Republic,B11,Inflows of foreign population,2015.0,9.0
152285,KOR,Korea,MRT,Mauritania,B11,Inflows of foreign population,2000.0,1.0
98254,KOR,Korea,GEO,Georgia,B11,Inflows of foreign population,2013.0,12.0


In [ ]:
inflows_pivot = inflow_df.pivot_table(
    index="Reference area",
    columns="TIME_PERIOD",
    values="OBS_VALUE",
    aggfunc="sum")

In [21]:
inflows_pivot.sample(5)

TIME_PERIOD,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Reference area,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Türkiye,192252.0,247872.0,309927.0,346215.0,64466.0,69137.0,66425.0,67520.0,70905.0,73435.0,84917.0,104023.0,NaN,33562.0,764.0,59494.0,NaN,NaN,NaN,NaN,NaN,544433.0,725398.0,930763.0,1153793.0,482389.0,1227522.0,796057.0
Poland,NaN,NaN,NaN,9984.0,33511.0,30784.0,41686.0,58781.0,58881.0,71829.0,76328.0,66926.0,80235.0,83009.0,82514.0,82084.0,82648.0,94240.0,93127.0,63611.0,171321.0,213349.0,255306.0,274414.0,326493.0,326308.0,447989.0,335289.0
Hungary,28546.0,28318.0,27472.0,32751.0,41112.0,41157.0,41242.0,36135.0,38969.0,43124.0,58867.0,1609064.0,46520.0,77020.0,56549.0,47596.0,44819.0,40416.0,42246.0,51781.0,51350.0,47429.0,72592.0,98236.0,110150.0,87274.0,97779.0,110668.0
Austria,NaN,109589.0,108890.0,130401.0,158025.0,145371.0,166071.0,186886.0,203698.0,227966.0,217346.0,188328.0,208136.0,215316.0,206624.0,192704.0,218720.0,250004.0,268858.0,306630.0,395310.0,316106.0,277447.0,262374.0,268800.0,241836.0,278031.0,491244.0
Australia,152336.0,196746.0,208183.0,202462.0,215229.0,233346.0,275492.0,258947.0,272039.0,327074.0,360222.0,392225.0,417495.0,447749.0,481569.0,440981.0,413057.0,472193.0,489847.0,467992.0,447653.0,437117.0,448679.0,373460.0,325077.5,274860.0,307586.0,434740.0


In [15]:
outflow_df = df_cleaned[df_cleaned['Measure'] == 'Outflows of foreign population']
outflow_df.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
6802,NZL,New Zealand,SVN,Slovenia,B12,Outflows of foreign population,2012.0,25.0
57807,HUN,Hungary,DEU,Germany,B12,Outflows of foreign population,2001.0,161.0
27507,KOR,Korea,HRV,Croatia,B12,Outflows of foreign population,2020.0,78.0
14405,AUT,Austria,LSO,Lesotho,B12,Outflows of foreign population,2019.0,1.0
14256,FIN,Finland,JAM,Jamaica,B12,Outflows of foreign population,2012.0,0.0


In [ ]:
outflows_pivot = outflow_df.pivot_table(
    index="Reference area",
    columns="TIME_PERIOD",
    values="OBS_VALUE",
    aggfunc="sum"
)

In [24]:
outflows_pivot.sample(5)

TIME_PERIOD,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0
Reference area,,,,,,,,,,,,,,,,,,,,,,,,,,,,
United Kingdom,146000.0,152000.0,215077.0,203574.0,308381.0,318918.0,273907.0,241766.0,242374.0,217971.0,235371.0,279541.0,284000.0,426000.0,375000.0,437000.0,363000.0,318000.0,321000.0,326000.0,315000.0,323000.0,376000.0,203000.0,202000.0,NaN,NaN,NaN
Belgium,79015.0,77175.0,83131.0,88586.0,89368.0,86579.0,79177.0,78646.0,82250.0,90967.0,92665.0,94434.0,91682.0,86585.0,94780.0,86741.0,103977.0,118468.0,137508.0,128518.0,118467.0,122743.0,116568.0,112943.0,92539.0,103373.0,117305.0,NaN
Korea,NaN,NaN,NaN,NaN,NaN,177528.0,217771.0,226734.0,308687.0,301075.0,529165.0,348415.0,304124.0,419963.0,466929.0,392133.0,435375.0,580069.0,536161.0,541018.0,601970.0,649872.0,697347.0,730149.0,851023.0,723156.0,526466.0,NaN
New Zealand,13162.0,15417.0,17697.0,19316.0,19057.0,31248.0,54624.0,54502.0,68121.0,76733.0,76139.0,73266.0,74257.0,79184.0,82414.0,87594.0,90324.0,82947.0,74620.0,71412.0,73987.0,78339.0,90618.0,94130.0,110111.0,78606.0,88065.0,87423.0
Switzerland,157198.0,153450.0,150856.0,138433.0,137958.0,130882.0,122576.0,115260.0,107056.0,111014.0,116208.0,124850.0,134832.0,125089.0,129724.0,130913.0,127844.0,131499.0,139792.0,138075.0,146689.0,154950.0,157979.0,161289.0,160039.0,140311.0,148521.0,NaN
